In [1]:
%pip install nltk pandas transformers
%pip install SpeechRecognition pydub
%pip install torch torchvision torchaudio
%pip install tensorflow
%pip install tensorflow --upgrade



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [44 lines of output]
      Traceback (most recent call last):
        File "/Users/hoobot/VS Code/RetentionInsightEngine/.venv/lib/python3.12/site-packages/setuptools/_vendor/packaging/requirements.py", line 35, in __init__
          parsed = _parse_requirement(requirement_string)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "/Users/hoobot/VS Code/RetentionInsightEngine/.venv/lib/python3.12/site-packages/setuptools/_vendor/packaging/_parser.py", line 64, in parse_requirement
          return

In [2]:
# Import required libraries
import speech_recognition as sr
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from pydub import AudioSegment
import pandas as pd
from transformers import pipeline


/Users/hoobot/VS Code/RetentionInsightEngine/.venv/lib/python3.12/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
/Users/hoobot/VS Code/RetentionInsightEngine/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Ensure NLTK resources are downloaded
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/hoobot/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/hoobot/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Speech to Text

In [10]:
# Convert MP3 to WAV for easier processing by SpeechRecognition
def convert_audio(input_file):
    sound = AudioSegment.from_mp3(input_file)
    output_file = "converted_file.wav"
    sound.export(output_file, format="wav")
    return output_file

In [11]:
 # Transcribe audio to text
def transcribe_audio(file_path):
    recognizer = sr.Recognizer()
    with sr.AudioFile(file_path) as source:
        audio_data = recognizer.record(source)
        try:
            text = recognizer.recognize_google(audio_data)
            return text
        except sr.UnknownValueError:
            return "Google Speech Recognition could not understand audio"
        except sr.RequestError as e:
            return "Could not request results from Google Speech Recognition service; {0}".format(e)



## Text Analysis

In [6]:
# Initialize the summarization pipeline
# Function from Hugging Face 'transformers' leveraging an LLM from a high-level API
# summarizer = pipeline("summarization")

# Facebook AI transformer model (Bidirectional and Auto-Regressive Transformers)
# It was pre-trained from corrupting text with an arbitrary noising function and learning to reconstruct the original text
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Google AI (Text-to-Text Transfer Transformer)
# It was pre-trained on a multi-task mixture of unsupervised and supervised tasks, using a C4 (Colossal Clean Crawled Corpus) dataset.
# summarizer = pipeline("summarization", model="t5-small")


In [7]:

# Process the transcription and generate a summary of pain points
def generate_summary(text):
    # Here, we can preprocess the text to improve summarization quality
    # For MVP and proof of concept, we'll directly summarize the transcription
    summary = summarizer(text, max_length=130, min_length=25, do_sample=False)
    return summary[0]['summary_text']

In [8]:
# Main processing function
def process_audio_file(input_file):
    converted_file = convert_audio(input_file)
    transcription = transcribe_audio(converted_file)
    print("Transcription:", transcription)

    # Generate summary
    summary = generate_summary(transcription)
    print("Summary:", summary)

    # Optionally, save the transcription and summary to files
    with open("transcribed_text.txt", "w") as text_file:
        text_file.write(transcription)
    with open("summary_text.txt", "w") as summary_file:
        summary_file.write(summary)

## Driver Code

In [13]:
# Replace 'Interview_Audio.mp3' with the exact audio file
process_audio_file("Exit Interview Questions from We.mp3")

Transcription: super glad you're interested in working with we and helping to create conversations that matter and build the organizations that are worth showing up to I actually just finished talking with Tiffany hello Tiffany during an exit interview so the last time that will formally talk as a part of Tiffany's internship as the social media scientist of we and one of the questions that we always at the last question knowing that we're passionate about questions the last question that we always like to ask is how can I how can you get your next dream job or dream thing so Tiffany I just had that conversation but I want to ask now Tiffany thinking about your time working with me or last year what is one of the most valuable things that you feel like you're taking away into your next job and life was before I started this internship I don't want to say it was closed but a year later period in general just knowing how to communicate with other people and just having an open social ent